# Course

In [ ]:
import calendar
from pathlib import Path

import pandas as pd
import plotly.express as px

In [ ]:
course_code = pd.read_csv("course_code.csv")
# course_code

In [ ]:
path = Path("classmate")
courses = pd.read_csv(path / "courses.csv")
with pd.option_context("display.max_rows", None):
    display(courses)

In [ ]:
courses[["學年", "學期", "開課系所代號", "課程代碼"]] = courses["科號"].str.extract(
    r"(\d+)(\d{2})([A-Z]+) ?(\d+)", expand=True
)
courses[["課程", "Course"]] = courses["課程名稱"].str.split(n=1, expand=True)
courses["教師姓名"] = courses["教師"]
matches = courses["教師姓名"].str.extractall(r"([\u4e00-\u9fff]+) ?([A-Z,\s-]+)?")
matches.loc[matches[0] == "指導教授", 1] = "Advisor"
courses[["教師", "Instructor"]] = (
    matches.dropna().groupby(level=0).agg(lambda x: ", ".join(x))
)
courses.loc[courses["教師"].isna(), "教師"] = courses["教師姓名"]
mapping_dict = course_code.set_index("Code", drop=True).to_dict(orient="index")
courses["開課系所_info"] = courses["開課系所代號"].map(mapping_dict)
courses[["開課系所", "Course Offering Department", "所屬學院", "College"]] = courses[
    "開課系所_info"
].apply(pd.Series)
courses.drop(["開課系所_info"], axis=1, inplace=True)
# courses

In [ ]:
days = "MTWRFS"
periods = "1234n56789abc"

for day in days:
    for period in periods:
        day_period = f"{day}{period}"
        courses[day_period] = courses["上課時間"].str.contains(day_period)
# courses

In [ ]:
courses.to_csv(path / "courses_parsed.csv", index=False)

In [ ]:
course = "課程"
# course = "Course"

## Curriculum

In [ ]:
curriculum = {}
for day, day_abbr in zip(days, calendar.day_abbr):
    temp = {}
    for period in periods:
        day_period = f"{day}{period}"
        courses_day = courses[courses[day_period] == True][course].values
        if courses_day.size > 0:
            temp[period] = "\n".join(courses_day)
        else:
            temp[period] = ""
    curriculum[day_abbr] = temp
curriculum_df = pd.DataFrame(curriculum, index=list(periods))
curriculum_df.style.set_properties(**{"white-space": "pre-wrap"})

In [ ]:
px.treemap(
    courses,
    path=["學年", "學期", course],
    color="開課系所",
    hover_name=course,
    hover_data=["科號", "教師", "上課時間"],
    title="Courses",
).update_traces(marker=dict(cornerradius=5))

In [ ]:
px.bar(
    courses,
    x=course,
    y="修課人數",
    color="開課系所",
    hover_name=course,
    title="Enrollment by Course",
)

In [ ]:
courses = courses.sort_values("學年")

In [ ]:
px.bar(
    courses,
    x="學年",
    color="學期",
    hover_name=course,
    title="Courses by Year and Semester",
).update_layout(yaxis_title="修課數")

In [ ]:
px.bar(
    courses,
    x="開課系所",
    color="學年",
    hover_name=course,
    hover_data="學期",
    title="Courses by Department",
).update_layout(yaxis_title="修課數")